# Week 03 — Hypothesis Testing (가설검정: A/B)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 캠페인 A와 B의 전환율 차이는 우연이 아닌가?
- 전환율 외에 '기대 매출' 차이도 함께 판단해야 하는가?

## 2) Why this matters (Theory)
- p-value는 '우연으로 이런 차이가 나올 확률'을 의미(가설이 참일 확률이 아님)
- 표본 수가 크면 작은 차이도 유의해질 수 있음 → 효과 크기 필수
- 전환율(비율)은 비율 검정/CI로 접근, 매출은 부트스트랩이 실무적으로 안정

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 A/B conversion rate test (z-test approx)
✅ 비율 차이에 대한 z-test(근사)와 uplift를 계산합니다.

In [ ]:
ab = df[df['campaign'].isin(['A','B'])].copy()
a = ab[ab['campaign']=='A']['conversions']
b = ab[ab['campaign']=='B']['conversions']

p1, p2 = a.mean(), b.mean()
n1, n2 = len(a), len(b)
p_pool = (a.sum()+b.sum())/(n1+n2)
se = np.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2))
z = (p1-p2)/se
pval = 2*(1-stats.norm.cdf(abs(z)))

uplift = (p1/p2 - 1) if p2>0 else np.nan
print("conv A:", p1, "conv B:", p2)
print("z:", z, "p-value:", pval)
print("uplift:", uplift)

In [ ]:
one_sentence_insight("p-value와 uplift를 함께 보고 '롤아웃 가치가 있는지' 한 문장")

### 5.2 Revenue impact via bootstrap (mean revenue per row)
✅ 전환이 0이 많아 비대칭 → 부트스트랩으로 평균 차이 CI를 추정합니다.

In [ ]:
ra = ab[ab['campaign']=='A']['revenue']
rb = ab[ab['campaign']=='B']['revenue']
mean_diff, ci = bootstrap_mean_diff(ra, rb, iters=4000, seed=0)
print("mean revenue diff (A - B):", mean_diff)
print("95% CI:", ci)

In [ ]:
one_sentence_insight('매출 기준으로 A가 B보다 좋은지(그리고 확신할 수 있는지) 한 문장')

### 5.3 Guardrail: margin impact check
✅ 매출이 늘어도 마진이 악화되면 실패 실험일 수 있습니다.

In [ ]:
ma = ab[ab['campaign']=='A']['gross_margin']
mb = ab[ab['campaign']=='B']['gross_margin']
m_diff, m_ci = bootstrap_mean_diff(ma, mb, iters=4000, seed=1)
print("mean margin diff (A - B):", m_diff)
print("95% CI:", m_ci)

In [ ]:
one_sentence_insight('전환/매출/마진을 종합했을 때 최종 추천 액션을 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
